In [1]:
import numpy as np
import pandas as pd

import pydicom
import glob

train_df = pd.read_csv("../input/vinbigdata-chest-xray-abnormalities-detection/train.csv")
train_df = train_df.query("class_id==3 and rad_id == 'R8' ").reset_index(drop = True)
train_df = train_df.drop_duplicates(subset=['image_id']).reset_index(drop = True)
train_df.tail()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max
1758,a914589fe6b948522475f7bf7e7b1136,Cardiomegaly,3,R8,823.0,1256.0,1793.0,1630.0
1759,e6a9b0eb532d8aab0ce29b787d81614b,Cardiomegaly,3,R8,614.0,1086.0,1536.0,1511.0
1760,1691ab3a82040297355b59a34c1e3fa9,Cardiomegaly,3,R8,709.0,1194.0,1430.0,1523.0
1761,eee88aa055892224d0fa0518f304996b,Cardiomegaly,3,R8,966.0,1710.0,2105.0,2099.0
1762,4b56bc6d22b192f075f13231419dfcc8,Cardiomegaly,3,R8,771.0,979.0,1680.0,1311.0


In [2]:
meta_data = pd.read_csv("/kaggle/input/meta-data-diplom/meta_data.csv")
meta_data = meta_data.query("class_id==3 and rad_id == 'R8' ").reset_index(drop = True)
meta_data = meta_data.drop_duplicates(subset=['image_id']).reset_index(drop = True)
meta_data.tail()

,Unnamed: 0,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,sex,age,weight,size,bmi,allergies,smoking,pregnancy,height,width
1758,67578,a914589fe6b948522475f7bf7e7b1136,Cardiomegaly,3,R8,823.0,1256.0,1793.0,1630.0,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2880,2304
1759,67684,e6a9b0eb532d8aab0ce29b787d81614b,Cardiomegaly,3,R8,614.0,1086.0,1536.0,1511.0,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2880,2304
1760,67726,1691ab3a82040297355b59a34c1e3fa9,Cardiomegaly,3,R8,709.0,1194.0,1430.0,1523.0,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2128,2152
1761,67745,eee88aa055892224d0fa0518f304996b,Cardiomegaly,3,R8,966.0,1710.0,2105.0,2099.0,M,071Y,NaN,NaN,NaN,NaN,NaN,NaN,2738,2486
1762,67912,4b56bc6d22b192f075f13231419dfcc8,Cardiomegaly,3,R8,771.0,979.0,1680.0,1311.0,F,052Y,NaN,NaN,NaN,NaN,NaN,NaN,2040,1968


In [3]:
train_df['width'] = meta_data['width']
train_df['height'] = meta_data['height']

In [ ]:
#crop image
train_df['height'] = train_df.apply(lambda row: int(0.96 * row.height), axis =1)
train_df.tail()

In [4]:
train_df['x_min'] = train_df.apply(lambda row: row.x_min/row.width, axis =1)
train_df['y_min'] = train_df.apply(lambda row: row.y_min/row.height, axis =1)

train_df['x_max'] = train_df.apply(lambda row: row.x_max/row.width, axis =1)
train_df['y_max'] = train_df.apply(lambda row: row.y_max/row.height, axis =1)

train_df['x_mid'] = train_df.apply(lambda row: (row.x_max+row.x_min)/2, axis =1)
train_df['y_mid'] = train_df.apply(lambda row: (row.y_max+row.y_min)/2, axis =1)

train_df['w'] = train_df.apply(lambda row: (row.x_max-row.x_min), axis =1)
train_df['h'] = train_df.apply(lambda row: (row.y_max-row.y_min), axis =1)

train_df['area'] = train_df['w']*train_df['h']
train_df.tail()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,width,height,x_mid,y_mid,w,h,area
1758,a914589fe6b948522475f7bf7e7b1136,Cardiomegaly,3,R8,0.357205,0.436111,0.778212,0.565972,2304,2880,0.567708,0.501042,0.421007,0.129861,0.054672
1759,e6a9b0eb532d8aab0ce29b787d81614b,Cardiomegaly,3,R8,0.266493,0.377083,0.666667,0.524653,2304,2880,0.466580,0.450868,0.400174,0.147569,0.059053
1760,1691ab3a82040297355b59a34c1e3fa9,Cardiomegaly,3,R8,0.329461,0.561090,0.664498,0.715695,2152,2128,0.496980,0.638393,0.335037,0.154605,0.051799
1761,eee88aa055892224d0fa0518f304996b,Cardiomegaly,3,R8,0.388576,0.624543,0.846742,0.766618,2486,2738,0.617659,0.695581,0.458166,0.142075,0.065094
1762,4b56bc6d22b192f075f13231419dfcc8,Cardiomegaly,3,R8,0.391768,0.479902,0.853659,0.642647,1968,2040,0.622713,0.561275,0.461890,0.162745,0.075170


In [5]:
from tqdm import tqdm

!mkdir class3_rad8
folder_location = "class3_rad8"

train_df['yolo_box_labels'] = train_df[['x_mid', 'y_mid', 'w', 'h']].values.tolist()

unique_img_ids = train_df.image_id.unique()
for img_id in tqdm(unique_img_ids): 
    patient_records = train_df.query("image_id == @img_id") 

    with open(folder_location + "/" + img_id + ".txt", 'w+') as file:
        for i in patient_records.iterrows():
            file.write(f"0 %s %s %s %s \n" % tuple(i[1].yolo_box_labels))

100%|██████████| 1763/1763 [00:04<00:00, 393.62it/s]


In [6]:
all_data = pd.read_csv("../input/vinbigdata-chest-xray-abnormalities-detection/train.csv")
without_disease = all_data.query("class_id==14 and rad_id == 'R8'").reset_index(drop = True)
without_disease_img_ids = without_disease.drop_duplicates(subset=['image_id']).reset_index(drop = True).image_id.unique()

for img_id in list(without_disease_img_ids):
    with open(folder_location + "/" + img_id + ".txt", 'w') as fp:
        pass

In [7]:
%%capture

# zip to make files easier to download

!zip -r class3_rad8.zip class3_rad8

!rm -r class3_rad8